# **Doublet-Source Panel Method**

In [1]:
include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")
using Revise
using .PanelMethods: Uniform2D, doublet_source_panels, collocation_point, split_panels, grid_data, aero_coefficients, pressure_coefficient, ×
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST, naca4, linspace
using BenchmarkTools
using Plots
plotly();

In [2]:
alpha_u = [0.2, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.2, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
airfoil = kulfan_CST(alphas, dzs, 0.1, 200);

In [3]:
uniform = Uniform2D(1.0, 5.0)
panels = doublet_source_panels(airfoil);

In [4]:
@time cl, cps = aero_coefficients(panels, uniform)
println("Lift Coefficient: $cl")

  2.540668 seconds (9.02 M allocations: 470.055 MiB, 10.29% gc time)
Lift Coefficient: 0.813789748969543


In [5]:
x_domain, y_domain = (-1, 2), (-1, 1)
grid_size = 50
x_dom, y_dom = linspace(x_domain..., grid_size), linspace(y_domain..., grid_size)
grid = x_dom × y_dom

vels, pots = grid_data(panels, grid)

cp = pressure_coefficient.(vels, uniform.mag);

LoadError: MethodError: no method matching /(::Main.PanelMethods.DoubletSourcePanel2D, ::Float64)
Closest candidates are:
  /(!Matched::Missing, ::Number) at missing.jl:115
  /(!Matched::BigFloat, ::Union{Float16, Float32, Float64}) at mpfr.jl:437
  /(!Matched::Float64, ::Float64) at float.jl:407
  ...

In [6]:
plot(airfoil[:,1], airfoil[:,2], markershape = :circle, aspect_ratio = :equal)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
upper_panels, lower_panels = split_panels(panels)
plot([ (first ∘ collocation_point)(panel) for panel in upper_panels ], [ panel.cp for panel in upper_panels ], 
     label = "Upper", markershape = :circle)
plot!([ (first ∘ collocation_point)(panel) for panel in lower_panels ], [ panel.cp for panel in lower_panels ], 
      label = "Lower", markershape = :circle, yaxis = (:flip))

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
p1 = contour(x_dom, y_dom, cp, fill = true)
plot(p1)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), color = "black", aspect_ratio = :equal)

LoadError: MethodError: no method matching cp(::Float64, ::Float64)